In [1]:
%cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [2]:
#import libraries
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#load word embedding model
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 958.5/958.4MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def loadTextFiles(a,b): #two text files were taken and they were the same
  global doc1, doc2
  with open(a, 'r') as file:
      doc1 = file.read().replace('\n', ' ')
  with open(b, 'r') as file:
      doc2 = file.read().replace('\n', ' ')
  documents = [doc1,doc2]
  return documents

In [0]:
def createDocumentTermMatrix(documents):
    # Create the Document Term Matrix
    #CountVectorizer tokenizes text files and builds a vocabulary of known words. We passed in "stop_words" to exclude the common stop words such as "a,is,the" etc.
    count_vectorizer = CountVectorizer(stop_words='english')
    count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(documents)
    showDocumentMatrix(sparse_matrix,count_vectorizer)
#This function transfors our sparse matrix to a dense matrix. This means that our training data will exclude non-zero values
#A sparse matrix has more zero values and a dense matrix has least zero values
def showDocumentMatrix(sparse_matrix, count_vectorizer):   
    doc_term_matrix = sparse_matrix.todense()
    global df
    df = pd.DataFrame(doc_term_matrix, 
                      columns=count_vectorizer.get_feature_names(), 
                      index=['doc_1', 'doc_2'])
    print(df)

In [0]:
#Get cosine similarity using Sklearn built-in function
def getCosineSimilarity(df):
    print(cosine_similarity(df)[0][1])

#Self defined function to get cosine similarity
def stem_tokens(tokens):
    '''This function is used to generalise the words by removing
    suffixes For example: If we did NOT use this function, words like "playing,play and player will" will be considered as different words
    This function removes the suffixes and looks at all of those words as one word "play" '''
    stemmer = nltk.stem.porter.PorterStemmer()
    return [stemmer.stem(item) for item in tokens]

'''
In this function, we are tokenizing the texts that we have. We used text.lower() to make sure all of the words are treated the same.
I.E - "The", "thE" and "tHe" are now just "the" 
'''
def normalize(text):
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    return stem_tokens(nltk.word_tokenize(text.lower()))

'''
This is where we pass our two text files to find the cosine similarity
We use Tfidf Vectorizer transforms the texts to Vectors which we need to use in order to use our Cosine Formula.
The cosine formula is the same as the one we use in maths and applies to vectors
'''
def getSelfCosineSimilarity(doc1, doc2):
    vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')
    tfidf = vectorizer.fit_transform([doc1, doc2])
    print(((tfidf * tfidf.T).A)[0,1])

In [0]:
#Soft Cosine Similarity
def dictionaryCorpus(documents):
    global dictionary
    dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

    
 #Here we basically are using the trained data from our model to find similarities with the data from our text files. For example:
    #Say our trained model has the word "boat" in it. And our text file has the word "ship". This will find a pattern between these two words.
def similarityMatrix(dictionary):
    similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)
    return similarity_matrix

def convertToBagOfWords(dictionary,doc1,doc2):
    sent_1 = dictionary.doc2bow(simple_preprocess(doc1))
    sent_2 = dictionary.doc2bow(simple_preprocess(doc2))
    global sentences
    sentences = [sent_1, sent_2]

In [0]:
def softCosineSim(sent_1,sent_2,similarity_matrix):
    print(softcossim(sent_1, sent_2, similarity_matrix))

In [9]:
#put in the name of the two text files that you want to read
documents = loadTextFiles('doc1.txt','doc2.txt')
createDocumentTermMatrix(documents)

       60  accuracy  almost  and  assumption  average  ...  x1  x2  x3  x4  xn  zero
doc_1   2         1       1    1           1        1  ...   4   4   2   2   1     1
doc_2   2         1       1    1           1        1  ...   4   4   2   2   1     1

[2 rows x 103 columns]


In [10]:
#Using cosine similarity to find similarity

print("Using Sklearn our cosine similarity value is: ",end='')
getCosineSimilarity(df)
print("Using a self-built function with NLP tools our cosine similarity value is: ",end='')
getSelfCosineSimilarity(doc1,doc2)

Using Sklearn our cosine similarity value is: 0.9999999999999987
Using a self-built function with NLP tools our cosine similarity value is: 1.0000000000000002


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_wor

In [11]:
#soft cosine similarity
dictionaryCorpus(documents)
similarity_matrix = similarityMatrix(dictionary)
convertToBagOfWords(dictionary,doc1,doc2)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [12]:
print("Using Soft Cosine Similarity our value is: ",end='')
softCosineSim(sentences[0],sentences[1],similarity_matrix)

Using Soft Cosine Similarity our value is: 1.0
